In [2]:
import pandas as pd
import time
import random

from bs4 import BeautifulSoup as bs
import requests
import numpy as np

from IPython.display import display
from datetime import datetime

from dotenv import load_dotenv
import os
load_dotenv()

frescos = {
    # "Frutas": "https://www.alcampo.es/compra-online/frescos/frutas/c/W1701?q=%3Arelevance&page=",
    # "Carne": "https://www.alcampo.es/compra-online/frescos/carne/c/W13?q=%3Arelevance&page=",
    # "Panadería":"https://www.alcampo.es/compra-online/frescos/panaderia/c/W1281?q=%3Arelevance&page=",
    # "Verduras y hortalizas":"https://www.alcampo.es/compra-online/frescos/verduras-y-hortalizas/c/W1702?q=%3Arelevance&page=",
    # "Charcutería":"https://www.alcampo.es/compra-online/frescos/charcuteria/c/W15?q=%3Arelevance&page=",
    # "Pastelería":"https://www.alcampo.es/compra-online/frescos/pasteleria/c/W1282?q=%3Arelevance&page=",
    # "Sushi": "https://www.alcampo.es/compra-online/frescos/sushi/c/W140502?q=%3Arelevance&page=",
    "Quesos": "https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=",
    "Pescados, mariscos y moluscos": "https://www.alcampo.es/compra-online/frescos/pescados-mariscos-y-moluscos/c/W14?q=%3Arelevance&page=",
    "Ahumados, sucedáneos gulas, anchoas": "https://www.alcampo.es/compra-online/frescos/ahumados-sucedaneos-gulas-anchoas/c/W184?q=%3Arelevance&page=",
    "Fresco al Corte": "https://www.alcampo.es/compra-online/frescos/fresco-al-corte/c/WFrescoalcorte?q=%3Arelevance&page="
}



dict_alcampo = {"Category": [], "Name":[],"Price":[],"Stock":[], "Image":[]}

for i in frescos.items():

    categoria = i[0]
    url_to_scrape = i[1]
    contador = 0
    product_not_found = 10

    while product_not_found != "0":
        
        url = url_to_scrape+ f"{contador}"
        response = requests.get(url='https://proxy.scrapeops.io/v1/',params={'api_key': os.environ["SCARPE_OPS_API_KEY"],'url': url, },)
        html = response.content
        soup = bs(html,"lxml")
     
        print(categoria,url)
        try:
            product_not_found = soup.find("div",class_="totalResults").get_text(strip=True, separator='').split(" ")[0]
        except:
            print("Got an attribute error on:",url)

        if product_not_found =="0":
            print("No hay más ventanas")
            break
        
        else:
            products = soup.find_all("div",class_="productGridItemContainer")
            prices = soup.find_all("div",class_="priceContainer")
            stock_available = soup.find_all("div",class_="cart add")

            for product in products:

                dict_alcampo["Name"].append(product.find("span").get_text(strip=True))
                dict_alcampo["Image"].append(product.find("img")["src"])

                dict_alcampo["Category"].append(categoria)

            for price in prices:
                try:
                    dict_alcampo["Price"].append(price.find_all("span")[2].get_text(strip=True))
                except:
                    print("List index out of range")
                    dict_alcampo["Price"].append("NaN")

            for stock in stock_available:
                
                stock_yes_no = stock.find_all("button")[2].get_text(strip=True).split(" "[0])
                
                if stock_yes_no[0]=="Sin":
                    dict_alcampo["Stock"].append("No")
                else:
                    dict_alcampo["Stock"].append("Yes")
                              
            print( "Category:", len(dict_alcampo["Category"]), "Name:", len(dict_alcampo["Name"]) ,"Image:",len(dict_alcampo["Image"]),"Price:", len(dict_alcampo["Price"]), "Stock",len(dict_alcampo["Stock"])    )
            
        contador+=1

df_alcampo = pd.DataFrame(dict_alcampo)

df_alcampo



Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=0
Category: 48 Name: 48 Image: 48 Price: 48 Stock 48
Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=1
Category: 96 Name: 96 Image: 96 Price: 96 Stock 96
Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=2
Got an attribute error on: https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=2
Category: 96 Name: 96 Image: 96 Price: 96 Stock 96
Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=3
Category: 144 Name: 144 Image: 144 Price: 144 Stock 144
Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=4
Category: 192 Name: 192 Image: 192 Price: 192 Stock 192
Quesos https://www.alcampo.es/compra-online/frescos/quesos/c/WQuesos?q=%3Arelevance&page=5
Category: 240 Name: 240 Image: 240 Price: 240 Stock 240
Quesos https://www.alc

,Category,Name,Price,Stock,Image
0,Quesos,Queso de untar formato familiar PHILADELPHIA O...,"3,85 €(11,00 €/Kg)",Yes,https://www.alcampo.es/media/h0f/h5b/108269413...
1,Quesos,Queso rallado 3 quesos 100 % natural EL CASERI...,"2,53 €(21,08 €/Kg)",Yes,https://www.alcampo.es/media/h2f/hdc/109960656...
2,Quesos,Queso rallado especial pasta TGT HOCHLAND 100 g.,"1,45 €(14,50 €/Kg)",Yes,https://www.alcampo.es/media/h3e/h54/109756820...
3,Quesos,Queso mozzarella rallado especial pizzas PRODU...,"1,58 €(7,90 €/Kg)",Yes,https://www.alcampo.es/media/h73/he6/966486609...
4,Quesos,Queso en lonchas Gouda PRODUCTO ALCAMPO 200 g.,"1,72 €(8,60 €/Kg)",Yes,https://www.alcampo.es/media/h91/hd0/946293417...
...,...,...,...,...,...
1265,Fresco al Corte,"Salchichón ibérico extra, cortado en lonchas E...","4,98 €(19,90 €/Kg)",Yes,https://www.alcampo.es/media/h82/h56/121031055...
1266,Fresco al Corte,Chicharro mostrador,"5€(9,99 €/Kg)",Yes,https://www.alcampo.es/media/h55/hb1/110263468...
1267,Fresco al Corte,Costillar de bellota ibérico de cerdo ALCAMPO ...,"2,24 €(8,95 €/Kg)",Yes,https://www.alcampo.es/media/h83/h03/109049182...
1268,Fresco al Corte,Chorizo ibérico cular,"4,98 €(19,90 €/Kg)",Yes,https://www.alcampo.es/media/hee/h4d/121030713...


In [3]:
df_alcampo['Shop'] = 'Alcampo'

today = pd.Timestamp.now()
df_alcampo['Date'] = today
df_alcampo

df_alcampo.to_csv("../data/alcampo.csv", mode='a', index=False, header=False)

read_file = pd.read_csv ("../data/alcampo.csv")
read_file.to_excel ("../data/alcampo.xlsx", index = None, header=True)

alcampo = pd.read_csv("../data/alcampo.csv")
print("File saved",today)
alcampo

File saved 2023-03-09 17:52:59.628799


,Category,Name,Price,Stock,Image,Shop,Date
0,Veganos,Bebida de avena con alto contenido de calcio y...,"5,94 €(0,99 €/Litro)",Yes,https://www.alcampo.es/media/hbc/hc7/114853804...,Alcampo,2023-03-08 19:51:25.159831
1,Veganos,"Bebida de soja sin gluten, enriquecida con cal...","5,34 €(0,89 €/Litro)",Yes,https://www.alcampo.es/media/h2b/hc3/107936543...,Alcampo,2023-03-08 19:51:25.159831
2,Veganos,Bebida de avena con alto contenido de calcio y...,"0,99 €(0,99 €/Litro)",Yes,https://www.alcampo.es/media/hc3/h79/109357267...,Alcampo,2023-03-08 19:51:25.159831
3,Veganos,"Bebida de almendras 100% vegetal, sin azúcar n...","2,10 €(2,10 €/Litro)",Yes,https://www.alcampo.es/media/h31/h28/114171706...,Alcampo,2023-03-08 19:51:25.159831
4,Veganos,Bebida de avena ALCAMPO ECOLÓGICO brick de 1 l...,"11,22 €(0,18 €/100ml)",Yes,https://www.alcampo.es/media/hb5/h66/111936354...,Alcampo,2023-03-08 19:51:25.159831
...,...,...,...,...,...,...,...
23364,Fresco al Corte,"Salchichón ibérico extra, cortado en lonchas E...","4,98 €(19,90 €/Kg)",Yes,https://www.alcampo.es/media/h82/h56/121031055...,Alcampo,2023-03-09 17:52:59.628799
23365,Fresco al Corte,Chicharro mostrador,"5€(9,99 €/Kg)",Yes,https://www.alcampo.es/media/h55/hb1/110263468...,Alcampo,2023-03-09 17:52:59.628799
23366,Fresco al Corte,Costillar de bellota ibérico de cerdo ALCAMPO ...,"2,24 €(8,95 €/Kg)",Yes,https://www.alcampo.es/media/h83/h03/109049182...,Alcampo,2023-03-09 17:52:59.628799
23367,Fresco al Corte,Chorizo ibérico cular,"4,98 €(19,90 €/Kg)",Yes,https://www.alcampo.es/media/hee/h4d/121030713...,Alcampo,2023-03-09 17:52:59.628799
